In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
df = pd.read_csv("cust_satisfaction.csv")
df.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied,13,460,5,4,5,25,18.0
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied,25,235,1,3,1,1,6.0
2,Female,Loyal Customer,Business travel,Business,satisfied,26,1142,5,4,5,0,0.0
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied,25,562,2,3,2,11,9.0
4,Male,Loyal Customer,Business travel,Business,satisfied,61,214,3,4,3,0,0.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      103904 non-null  object 
 1   Customer Type               103904 non-null  object 
 2   Type of Travel              103904 non-null  object 
 3   Class                       103904 non-null  object 
 4   satisfaction                103904 non-null  object 
 5   Age                         103904 non-null  int64  
 6   Flight Distance             103904 non-null  int64  
 7   Inflight entertainment      103904 non-null  int64  
 8   Baggage handling            103904 non-null  int64  
 9   Cleanliness                 103904 non-null  int64  
 10  Departure Delay in Minutes  103904 non-null  int64  
 11  Arrival Delay in Minutes    103594 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 9.5+ MB


In [12]:
df.isnull().sum()
df.dropna(inplace=True)

In [13]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [14]:
df["Customer Type"].value_counts()

Customer Type
Loyal Customer       84517
disloyal Customer    18905
Name: count, dtype: int64

In [15]:
loyal_customer = df[df["Customer Type"] == "Loyal Customer"]
disloyal_customer = df[df["Customer Type"] == "disloyal Customer"]

In [16]:
loyal_customer = loyal_customer.sample(20000)
loyal_customer.shape

(20000, 12)

In [17]:
balance_df = pd.concat([loyal_customer, disloyal_customer], axis=0)
balance_df.shape

(38905, 12)

In [18]:
cat_col = balance_df.select_dtypes(include=["object"])
cat_col.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
68233,Male,Loyal Customer,Business travel,Business,satisfied
61360,Male,Loyal Customer,Business travel,Eco,satisfied
65553,Male,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
24481,Female,Loyal Customer,Business travel,Business,satisfied
73968,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied


In [19]:
num_col = balance_df.select_dtypes(exclude=["object"])
num_col.head()

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
68233,31,3991,2,4,2,0,0.0
61360,50,192,4,5,4,6,0.0
65553,52,723,3,4,3,22,15.0
24481,49,1504,4,4,2,85,66.0
73968,22,134,3,2,3,16,5.0


In [20]:
pd.get_dummies(cat_col, drop_first=True).astype(int).head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
68233,1,0,0,0,0,1
61360,1,0,0,1,0,1
65553,1,0,1,1,0,0
24481,0,0,0,0,0,1
73968,0,0,1,1,0,0


In [21]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohe = OneHotEncoder(drop="if_binary")
cat_col_encoded = ohe.fit_transform(cat_col).toarray()
cat_col_encoded

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [22]:
column_name = list(ohe.get_feature_names_out())
column_name

['Gender_Male',
 'Customer Type_disloyal Customer',
 'Type of Travel_Personal Travel',
 'Class_Business',
 'Class_Eco',
 'Class_Eco Plus',
 'satisfaction_satisfied']

In [23]:
one_hot = pd.DataFrame(cat_col_encoded, columns=column_name)
one_hot.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [24]:
one_hot = one_hot.reset_index(drop=True)
num_col = num_col.reset_index(drop=True)
final_df = pd.concat([one_hot, num_col], axis=1)
final_df.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,31,3991,2,4,2,0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,50,192,4,5,4,6,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,52,723,3,4,3,22,15.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,49,1504,4,4,2,85,66.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,22,134,3,2,3,16,5.0


In [25]:
##x and y ---> train test split -->algo train
x = final_df.drop("Customer Type_disloyal Customer", axis=1)
y = final_df["Customer Type_disloyal Customer"]

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [27]:
## standard scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [29]:
x_train.shape[1]

13

In [31]:
##define your ann model 
model = Sequential()
##input layer 
##68 -- no of neurons -- general convention -- 128 
model.add(Dense(68,activation="relu",input_dim = x_train_scaled.shape[1]))
##hidden layer 
model.add(Dense(32, activation="relu"))
model.add(Dense(24, activation="relu"))
model.add(Dense(12, activation="relu"))
##output layer 
model.add(Dense(1, activation="sigmoid"))
##model compile 
model.compile(optimizer="adam",
               loss="binary_crossentropy",
                 metrics=["accuracy"])
model.summary()

c:\Users\KHUSHI GUPTA\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 68)             │           952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,265 (16.66 KB)

 Trainable params: 4,265 (16.66 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
(13+1)* 68

(68+1) *32

2208

In [33]:
history = model.fit(x_train_scaled,y_train,
                    epochs = 10 ,
                    validation_data=(x_test_scaled,y_test))

Epoch 1/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8699 - loss: 0.3296 - val_accuracy: 0.9061 - val_loss: 0.2405
Epoch 2/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9022 - loss: 0.2415 - val_accuracy: 0.9113 - val_loss: 0.2307
Epoch 3/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9121 - loss: 0.2240 - val_accuracy: 0.9149 - val_loss: 0.2204
Epoch 4/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9149 - loss: 0.2168 - val_accuracy: 0.9193 - val_loss: 0.2175
Epoch 5/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9173 - loss: 0.2099 - val_accuracy: 0.9185 - val_loss: 0.2140
Epoch 6/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9192 - loss: 0.2074 - val_accuracy: 0.9207 - val_loss: 0.2083
Epoch 7/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9208 - loss: 0.2023 - val_accuracy: 0.9183 - val_loss: 0.2098
Epoch 8/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9240 - loss: 0.1949 - val_accuracy: 0.

In [34]:
## prediction 
y_pred = model.predict(x_test_scaled)
prediction_label = (y_pred > 0.5).astype(int).ravel()
prediction_label

244/244 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([0, 1, 1, ..., 0, 1, 0])

In [35]:
model.save("model_classfication.h5")